# **Retrieve Company and Country Information**

In [ ]:
!pip install openai langchain langchain-community langchain-openai langgraph python-dotenv pypdf requests pandas semantic-router pinecone-client serpapi google-search-results pycountry wikipedia-api

In [26]:
!pip install -qU "langchain-community>=0.2.11" tavily-python

In [ ]:
pip install -U langchain langchain-openai pinecone-client

In [4]:
import os
import json
import wikipediaapi
import pycountry


USER_AGENT = "MyCountryInfoScript/1.0 (habtamufeyera95@gmail.com)"


def get_country_info(country_name):
    """
    Fetches key facts about a country from Wikipedia, first by looking for structured sections,
    and then scanning the full text if necessary.

    Args:
        country_name (str): The name of the country to search for.

    Returns:
        dict: Dictionary containing country information.
    """
    # Initialize the Wikipedia API client with a custom User-Agent
    headers = {'User-Agent': USER_AGENT}
    wiki_wiki = wikipediaapi.Wikipedia('en', headers=headers)

    # Fetch the page for the country
    page = wiki_wiki.page(country_name)


    if not page.exists():
        print(f"Page for '{country_name}' does not exist on Wikipedia.")
        return None

    # Extract general country info
    country_info = {
        "Country": country_name,
        "Summary": page.summary[:2000],
        "Economy": None,
        "Culture": None,
        "Business Environment": None
    }


    for section in page.sections:
        title = section.title.lower()

        if "economy" in title:
            country_info["Economy"] = section.text[:2000]
        elif "culture" in title:
            country_info["Culture"] = section.text[:2000]
        elif any(keyword in title for keyword in ["business", "investment", "trade", "corporate"]):
            country_info["Business Environment"] = section.text[:2000]


    if not country_info["Economy"]:
        country_info["Economy"] = search_full_text_for_section(page.text, "economy")
    if not country_info["Culture"]:
        country_info["Culture"] = search_full_text_for_section(page.text, "culture")
    if not country_info["Business Environment"]:
        country_info["Business Environment"] = search_full_text_for_section(page.text, "business")


    for key in ["Economy", "Culture", "Business Environment"]:
        if country_info[key] is None:
            country_info[key] = "No relevant section found."

    return country_info

def search_full_text_for_section(full_text, keyword):
    """
    Scans the full text to find and return relevant information based on a keyword.

    Args:
        full_text (str): The full Wikipedia page text.
        keyword (str): The keyword to search for in the text.

    Returns:
        str: The relevant information extracted from the text.
    """
    start_index = full_text.lower().find(keyword)
    if start_index != -1:
        end_index = full_text.find("\n", start_index + 1000)
        return full_text[start_index:end_index].strip()
    return "Relevant information not found in full text."

def get_all_countries_info(json_file_path='files/all_countries_info.json'):
    """
    Fetches and saves information for all countries from Wikipedia into a single JSON file.

    Args:
        json_file_path (str): The file path where the JSON data will be saved.

    Returns:
        list: List of dictionaries containing country information.
    """

    os.makedirs(os.path.dirname(json_file_path), exist_ok=True)

    all_country_info = []


    country_names = [country.name for country in pycountry.countries]

    for country_name in country_names:
        print(f"Fetching data for {country_name}...")
        country_info = get_country_info(country_name)

        if country_info:
            all_country_info.append(country_info)

    # Save all data in one JSON file
    try:
        with open(json_file_path, 'w', encoding='utf-8') as f:
            json.dump(all_country_info, f, ensure_ascii=False, indent=4)
            print(f"All country data saved to {json_file_path}")
    except Exception as e:
        print(f"Error saving data: {e}")

    return all_country_info

if __name__ == "__main__":
    # Fetch and store all countries' info in one document
    all_countries_data = get_all_countries_info()


Fetching data for Aruba...
Fetching data for Afghanistan...
Fetching data for Angola...
Fetching data for Anguilla...
Fetching data for Åland Islands...
Fetching data for Albania...
Fetching data for Andorra...
Fetching data for United Arab Emirates...
Fetching data for Argentina...
Fetching data for Armenia...
Fetching data for American Samoa...
Fetching data for Antarctica...
Fetching data for French Southern Territories...
Fetching data for Antigua and Barbuda...
Fetching data for Australia...
Fetching data for Austria...
Fetching data for Azerbaijan...
Fetching data for Burundi...
Fetching data for Belgium...
Fetching data for Benin...
Fetching data for Bonaire, Sint Eustatius and Saba...
Fetching data for Burkina Faso...
Fetching data for Bangladesh...
Fetching data for Bulgaria...
Fetching data for Bahrain...
Fetching data for Bahamas...
Fetching data for Bosnia and Herzegovina...
Fetching data for Saint Barthélemy...
Fetching data for Belarus...
Fetching data for Belize...
Fetch

In [5]:
import wikipediaapi
import concurrent.futures
import json
import re

# Custom User-Agent string for Wikipedia API request
USER_AGENT = "MyCompanyInfoScript/1.0 (habtamufeyera95@gmail.com)"

def search_full_text_for_section(text, keyword):
    """
    Searches the full text for relevant information on a given keyword.

    Args:
        text (str): The full text of the Wikipedia page.
        keyword (str): The keyword to search for in the text.

    Returns:
        str: A substring of text (up to 1000 characters) starting at the keyword, or None.
    """
    lower_text = text.lower()
    lower_keyword = keyword.lower()
    if lower_keyword in lower_text:
        start_idx = lower_text.find(lower_keyword)
        end_idx = start_idx + 1000  # Limit to 1000 characters
        return text[start_idx:end_idx]
    return None

def extract_industry_from_summary(summary):
    """
    Attempts to extract an industry phrase from the page summary.
    Looks for a pattern like "is a ... company".

    Args:
        summary (str): The summary text of the Wikipedia page.

    Returns:
        str: The captured industry phrase or None.
    """
    # Pattern example: "Apple Inc. is a technology company" -> capture "technology"
    match = re.search(r"is a ([\w\s,&-]+?) (company|firm)", summary, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

def get_company_info(company_name):
    """
    Fetches key facts about a company from Wikipedia.
    It extracts:
      - Background (from a section like "History")
      - Industry (from a dedicated section, a full-text search, or from the summary)
      - Achievements (from sections like "Business strategy", "Technology", or "Achievements")
      - Impact (from sections like "Lawsuits and controversies", "Corporate affairs", or "Impact")

    Args:
        company_name (str): The name of the company to search for.

    Returns:
        dict: A dictionary containing the company’s information.
    """
    # Initialize Wikipedia API with language and user agent
    wiki_wiki = wikipediaapi.Wikipedia(language='en', user_agent=USER_AGENT)
    page = wiki_wiki.page(company_name)

    if not page.exists():
        return {
            "Company": company_name,
            "Background": "No relevant section found.",
            "Industry": "No relevant section found.",
            "Achievements": "No relevant section found.",
            "Impact": "No relevant section found.",
            "Error": "Page not found"
        }

    company_info = {
        "Company": company_name,
        "Background": None,
        "Industry": None,
        "Achievements": None,
        "Impact": None
    }

    # --- Extract Background ---
    background_keywords = ["history"]
    background_text = None
    for section in page.sections:
        if any(keyword in section.title.lower() for keyword in background_keywords):
            background_text = section.text[:2000]  # Limit to first 2000 characters
            break
    if not background_text:
        background_text = page.text[:2000]  # Fallback to the first part of the page text
    company_info["Background"] = background_text

    # --- Extract Industry ---
    industry_keywords = ["industry", "sector", "products", "services", "automotive"]
    industry_text = None
    for section in page.sections:
        if any(keyword in section.title.lower() for keyword in industry_keywords):
            industry_text = section.text[:2000]  # Limit to first 2000 characters
            break
    # If not found, try a full-text search for "industry"
    if not industry_text:
        industry_text = search_full_text_for_section(page.text, "industry")
    # As a last resort, attempt to extract from the summary
    if not industry_text:
        industry_text = extract_industry_from_summary(page.summary)
    company_info["Industry"] = industry_text or "No relevant section found."

    # --- Extract Achievements ---
    achievements_keywords = ["business strategy", "technology", "achievements"]
    achievements_text = None
    for section in page.sections:
        if any(keyword in section.title.lower() for keyword in achievements_keywords):
            achievements_text = section.text[:2000]
            break
    if not achievements_text:
        achievements_text = search_full_text_for_section(page.text, "achievements")
    company_info["Achievements"] = achievements_text or "No relevant section found."

    # --- Extract Impact ---
    impact_keywords = ["lawsuits and controversies", "corporate affairs", "impact"]
    impact_text = None
    for section in page.sections:
        if any(keyword in section.title.lower() for keyword in impact_keywords):
            impact_text = section.text[:2000]
            break
    if not impact_text:
        impact_text = search_full_text_for_section(page.text, "impact")
    company_info["Impact"] = impact_text or "No relevant section found."

    return company_info

def fetch_company_data_concurrently(company_names):
    """
    Fetches company data concurrently for multiple companies using threading.

    Args:
        company_names (list): List of company names.

    Returns:
        list: List of dictionaries with company information.
    """
    all_company_info = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_company_info, company_name) for company_name in company_names]
        for future in concurrent.futures.as_completed(futures):
            all_company_info.append(future.result())
    return all_company_info

if __name__ == "__main__":
    company_names = [
        "Apple", "Tesla", "Microsoft", "Google", "Amazon", "Facebook", "Netflix",
        "IBM", "Oracle", "Adobe", "Twitter", "Zoom", "Uber", "Airbnb", "Spotify",
        "Intel", "Nvidia", "Salesforce", "Snapchat", "LinkedIn", "Pinterest",
        "Lyft", "Slack", "Square", "Atlassian", "Shopify", "PayPal", "Stripe",
        "Dropbox", "Reddit", "Instacart", "Postmates", "Figma", "Robinhood",
        "Plaid", "ByteDance", "TikTok", "Alibaba", "Tencent", "Huawei", "Xiaomi",
        "Baidu", "JD.com", "Meituan", "Didi", "Zebra Technologies", "Palantir",
        "Databricks", "Snowflake", "Elastic", "MongoDB", "HashiCorp", "Twilio",
        "T-Mobile", "ZoomInfo", "Veeva", "Coupa", "Okta", "ServiceTitan",
        "Qualys", "Datadog", "Coupang", "Cognizant", "Accenture", "DXC Technology",
        "Intuit", "Cigna", "Aetna", "UnitedHealth", "CVS Health", "Merck", "Pfizer",
        "Eli Lilly", "Bristol-Myers Squibb", "AbbVie", "Johnson & Johnson",
        "Procter & Gamble", "Unilever", "Nestlé", "Coca-Cola", "PepsiCo",
        "General Electric", "Lockheed Martin", "Raytheon", "Ethio Telecom",
        "Ethiopian Airlines", "DHL Ethiopia", "Bunna International Bank",
        "Commercial Bank of Ethiopia", "Zemen Bank", "Awash International Bank",
        "Dashen Bank", "Addis Ababa Light Rail", "Mulugeta Teshome Group"
    ]

    company_data = fetch_company_data_concurrently(company_names)

    # Save data to a JSON file
    json_data = json.dumps(company_data, indent=4)
    with open("company_info.json", "w") as json_file:
        json_file.write(json_data)


In [ ]:
import os
import logging
import numpy as np
import pandas as pd
import concurrent.futures
from typing import List, TypedDict, Dict, Any
from dotenv import load_dotenv

# ------------------------------
# LangChain & Vector Store Imports
# ------------------------------
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone

# ------------------------------
# Pinecone & External Client Imports
# ------------------------------
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from tavily import TavilyClient

# ------------------------------
# LangGraph for Workflow Orchestration
# ------------------------------
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# ------------------------------
# Environment Setup
# ------------------------------
load_dotenv()
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Validate API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

if not all([OPENAI_API_KEY, PINECONE_API_KEY, TAVILY_API_KEY]):
    raise ValueError("Missing API keys. Please check your environment variables.")

# Initialize Embeddings & LLM
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
model = ChatOpenAI(model="gpt-4", temperature=0)
tavily = TavilyClient(api_key=TAVILY_API_KEY)

# ------------------------------
# Pinecone Client Initialization
# ------------------------------
pc = PineconeClient(api_key=PINECONE_API_KEY)
index_name = "promptwithrag"

def manage_pinecone_index():
    """Ensure the Pinecone index exists and create it if necessary."""
    try:
        existing_indexes = pc.list_indexes().names()
        if index_name in existing_indexes:
            logging.info(f"Index '{index_name}' already exists.")
            return
        logging.info(f"Creating index '{index_name}'...")
        spec = ServerlessSpec(cloud="aws", region="us-east-1")
        pc.create_index(name=index_name, dimension=1536, metric="cosine", spec=spec)
        logging.info(f"Index '{index_name}' created successfully.")
    except Exception as e:
        logging.exception(f"Error managing Pinecone index: {e}")

manage_pinecone_index()
index = pc.Index(index_name)  

# ------------------------------
# Data Loading & Processing
# ------------------------------
def load_json_file(file_path: str) -> pd.DataFrame:
    """Loads a JSON file and returns a DataFrame."""
    if not os.path.exists(file_path):
        logging.error(f"File not found: {file_path}")
        return pd.DataFrame()

    try:
        df = pd.read_json(file_path)
        logging.info(f"Loaded {len(df)} records from {file_path}")
        return df
    except Exception as e:
        logging.exception(f"Error loading file {file_path}: {e}")
        return pd.DataFrame()

countries_df = load_json_file("/content/files/all_countries_info.json")
companies_df = load_json_file("/content/company_info.json")

def combine_text_columns(df: pd.DataFrame, expected_columns: List[str], new_col_name: str) -> pd.DataFrame:
    """Combine specified columns into a single text field for processing."""
    present_columns = [col for col in expected_columns if col in df.columns]
    if not present_columns:
        present_columns = df.select_dtypes(include="object").columns.tolist()
        logging.warning(f"Using all object columns for '{new_col_name}': {present_columns}")

    df[new_col_name] = df[present_columns].fillna("").agg(" ".join, axis=1)
    return df

countries_df = combine_text_columns(countries_df, ["Summary", "Economy", "Culture"], "country_description")
companies_df = combine_text_columns(companies_df, ["Background", "Industry", "Achievements", "Impact"], "company_description")

def chunk_documents(df: pd.DataFrame, text_column: str, source: str, chunk_size=500, chunk_overlap=50) -> List[Dict[str, Any]]:
    """Split documents into manageable chunks for vector storage."""
    result = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    for doc_idx, row in df.iterrows():
        text = row[text_column]
        if pd.isna(text) or len(text) < 100:
            continue  # Skip very short documents
        chunks = splitter.split_text(text)
        result.extend([{"doc_id": f"{source}_{doc_idx}", "chunk_id": i, "text": chunk, "source": source} for i, chunk in enumerate(chunks)])
    return result

all_chunks = chunk_documents(countries_df, "country_description", "country") + chunk_documents(companies_df, "company_description", "company")
df_chunks = pd.DataFrame(all_chunks)

# ------------------------------
# Generate Embeddings & Upsert to Pinecone
# ------------------------------
def upsert_embeddings():
    """Generate embeddings and store them in Pinecone."""
    chunk_texts = df_chunks["text"].tolist()
    chunk_embeddings = embedding_function.embed_documents(chunk_texts)  

    def process_batch(batch):
        try:
            index.upsert(vectors=batch)
            return True
        except Exception as e:
            logging.exception(f"Error upserting batch: {e}")
            return False

    upsert_data = [
        (f"{row['doc_id']}_{row['chunk_id']}", chunk_embeddings[idx],
         {"doc_id": row["doc_id"], "chunk_id": row["chunk_id"], "text": row["text"], "source": row["source"]})
        for idx, row in df_chunks.iterrows()
    ]

    batch_size = 50
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_batch, [upsert_data[i:i+batch_size] for i in range(0, len(upsert_data), batch_size)]))

    logging.info(f"Upsert completed with {sum(results)} successful batches.")

upsert_embeddings()

# ------------------------------
# Set Up Retriever
# ------------------------------
vectorstore = Pinecone.from_existing_index(index_name=index_name, embedding=embedding_function)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# ------------------------------
# Define Agent State
# ------------------------------
class AgentState(TypedDict):
    company_name: str
    country_name: str
    company_info: str
    country_info: str
    draft_speech: str
    revision_number: int
    max_revisions: int
    retrieved_info: List[str]


In [14]:
# Prompt templates.
COMPANY_INFO_PROMPT = """
Gather detailed information about {company_name}, including:
- Background
- Industry
- Achievements
- Impact
"""

COUNTRY_INFO_PROMPT = """
Provide key insights about {country_name}, including:
- Economic Landscape
- Cultural Significance
- Business Environment
"""

SPEECH_GENERATION_PROMPT = """
Create a compelling speech using the following information:

Company Information:
{company_info}

Country Information:
{country_info}

The speech should be formal, well-structured, and impactful for an invited guest.
"""

SPEECH_REVIEW_PROMPT = """
Review the speech and provide constructive feedback.
"""

def retrieve_company_info(state: AgentState) -> dict:
    query = f"{state['company_name']} company overview, achievements, industry, and impact"
    try:
        response = tavily.search(query=query, max_results=3)
        if "results" in response:
            info = "\n".join([r["content"] for r in response["results"]])
        else:
            info = "Company information could not be retrieved."
    except Exception as e:
        logging.error(f"Error retrieving company info: {e}")
        info = "Error occurred while retrieving company information."
    return {"company_info": info}

def retrieve_country_info(state: AgentState) -> dict:
    query = f"{state['country_name']} economic landscape, culture, and business environment"
    try:
        response = tavily.search(query=query, max_results=3)
        if "results" in response:
            info = "\n".join([r["content"] for r in response["results"]])
        else:
            info = "Country information could not be retrieved."
    except Exception as e:
        logging.error(f"Error retrieving country info: {e}")
        info = "Error occurred while retrieving country information."
    return {"country_info": info}

def retrieve_external_info(state: AgentState) -> dict:
    try:
        # Use the new invoke method instead of get_relevant_documents.
        company_retrieval = retriever.invoke(f"{state['company_name']} company background")
        country_retrieval = retriever.invoke(f"{state['country_name']} country economic landscape")
        retrieved_info = company_retrieval + country_retrieval
    except Exception as e:
        logging.error(f"Error retrieving external info: {e}")
        retrieved_info = []
    return {"retrieved_info": retrieved_info}

def generate_speech(state: AgentState) -> dict:
    prompt = SPEECH_GENERATION_PROMPT.format(
        company_info=state["company_info"],
        country_info=state["country_info"]
    )
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": "Generate the speech."},
    ]
    try:
        response = model(messages)
        draft_speech = response.content
    except Exception as e:
        logging.error(f"Error generating speech: {e}")
        draft_speech = "Error occurred during speech generation."
    return {"draft_speech": draft_speech, "revision_number": state["revision_number"] + 1}

def review_speech(state: AgentState) -> dict:
    messages = [
        {"role": "system", "content": SPEECH_REVIEW_PROMPT},
        {"role": "user", "content": state["draft_speech"]},
    ]
    try:
        response = model(messages)
        speech_review = response.content
    except Exception as e:
        logging.error(f"Error reviewing speech: {e}")
        speech_review = "Error occurred during speech review."
    return {"speech_review": speech_review}

# Decision node: determines if another revision is needed.
def should_continue(state: AgentState) -> dict:
    if state["revision_number"] > state["max_revisions"]:
        return {"next": END}
    else:
        return {"next": "generate_speech"}

# ------------------------------
# Build the Workflow Graph
# ------------------------------
builder = StateGraph(AgentState)
builder.add_node("fetch_company_info", retrieve_company_info)
builder.add_node("fetch_country_info", retrieve_country_info)
builder.add_node("retrieve_external_info", retrieve_external_info)
builder.add_node("generate_speech", generate_speech)
builder.add_node("review_speech", review_speech)
builder.add_node("should_continue", should_continue)

builder.set_entry_point("fetch_company_info")
builder.add_edge("fetch_company_info", "fetch_country_info")
builder.add_edge("fetch_country_info", "retrieve_external_info")
builder.add_edge("retrieve_external_info", "generate_speech")
builder.add_edge("generate_speech", "review_speech")
# Route to the decision node after reviewing the speech.
builder.add_edge("review_speech", "should_continue")

In [ ]:
# ------------------------------
# Compile Graph with Checkpointer and Run the Agent for American Company
# ------------------------------
# Enable checkpointing using MemorySaver to preserve the state of the agent
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)  # Enable checkpointing during graph compilation

if __name__ == "__main__":
    # Define initial state to kick-start the agent's workflow for Microsoft
    initial_state = {
        "company_name": "Microsoft Corporation",
        "country_name": "United States of America",
        "revision_number": 1,  
        "max_revisions": 3,  
        "thread_id": "session_003",  
        "company_info": "",
        "country_info": "",
        "draft_speech": "",
        "retrieved_info": []
    }

    try:
        # Invoke the graph with the initial state and provide a configurable thread ID for checkpointing
        output = graph.invoke(initial_state, config={"configurable": {"thread_id": "session_003"}})
        print("Agent Output:", output)  
    except Exception as e:
        print(f"Error invoking the agent: {e}")


Agent Output: {'company_name': 'Microsoft Corporation', 'country_name': 'United States of America', 'company_info': 'Microsoft Corporation (MSFT) Company Profile & Facts - Yahoo Finance Microsoft Corporation (MSFT) Microsoft Corporation Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. Additionally, this segment provides gaming, which includes Xbox hardware and content, and first- and third-party content; Xbox game pass and other subscriptions, cloud gaming, advertising, third-party disc royalties, and other cloud services; and search and news advertising, which includes Bing, Microsoft News and Edge, and third-party affiliates. Cor

In [23]:
print(output["draft_speech"])

Ladies and Gentlemen,

I stand before you today to discuss a company that has not only revolutionized the world of technology but has also played a pivotal role in shaping the global economy. That company is none other than Microsoft Corporation. 

Founded in the 1980s, Microsoft initially gained dominance in the operating systems market with MS-DOS and Windows. Today, it has evolved into a multifaceted corporation that develops and supports software, services, devices, and solutions worldwide. Its operations can be divided into three major business segments: productivity and business processes, intelligent cloud, and more personal computing. 

Microsoft's productivity and business processes segment offers a range of services, including office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot. It also provides gaming, which includes Xbox hardware and content, and first- and third-party content; Xbox game pass and other

In [ ]:

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)  # Enable checkpointing during graph compilation

if __name__ == "__main__":
    # Define initial state to kick-start the agent's workflow for Microsoft
    initial_state = {
        "company_name": "Apple Inc",
        "country_name": "United States of America",
        "revision_number": 1,  
        "max_revisions": 3,  
        "thread_id": "session_003",  
        "company_info": "",
        "country_info": "",
        "draft_speech": "",
        "retrieved_info": []
    }

    try:
        
        output = graph.invoke(initial_state, config={"configurable": {"thread_id": "session_003"}})
        print("Agent Output:", output)  
    except Exception as e:
        print(f"Error invoking the agent: {e}")


Agent Output: {'company_name': 'Apple Inc', 'country_name': 'United States of America', 'company_info': 'Apple Inc: About The Company - SimplyMac Apple Inc: About The Company Apple Inc. is a leading technology company based in Cupertino, California. The company’s product lineup includes iconic devices such as the iPhone, iPad, Mac computers, and Apple Watch. These products are known for their sleek design, user-friendly interfaces, and seamless integration with Apple’s software ecosystem. Apple Inc. is a multinational technology company. Apple prioritizes innovation, quality, and user experience in its products and services. Apple TV+ is the company’s streaming video service, featuring original content. Apple Inc. is a prominent technology company with a rich history and innovative products. What was the first product launched by Apple? The Apple I computer was the company’s first product.\nApple Inc. Founded by Steve Jobs and Steve Wozniak in 1976, Apple set new benchmarks in product 

In [25]:
print(output["draft_speech"])

Ladies and Gentlemen,

I am honored to stand before you today to discuss the remarkable journey of innovation and the relentless pursuit of excellence that defines our technology sector. We are here to celebrate the achievements of some of the most influential companies in the world, companies that have not only shaped the technology landscape but have also played a pivotal role in transforming our lives and the way we interact with the world.

Let us begin with Apple Inc., a company that needs no introduction. Founded in 1976 by Steve Jobs and Steve Wozniak, Apple has been a beacon of innovation, setting new benchmarks in product design, user-centric functionality, and multi-product integration. From the Apple I computer, their first product, to the iconic iPhone, iPad, Mac computers, and Apple Watch, Apple's products are renowned for their sleek design, user-friendly interfaces, and seamless integration with Apple’s software ecosystem. Apple's commitment to innovation, quality, and u

In [ ]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)  

if __name__ == "__main__":
    initial_state = {
        "company_name": "Ethiopian Airlines",
        "country_name": "Ethiopia",
        "revision_number": 1,
        "max_revisions": 2,
        "thread_id": "session_001",  
        "company_info": "",
        "country_info": "",
        "draft_speech": "",
        "retrieved_info": []
    }

    #
    output = graph.invoke(initial_state, config={"configurable": {"thread_id": "session_001"}})
    print(output)


{'company_name': 'Ethiopian Airlines', 'country_name': 'Ethiopia', 'company_info': 'ETHIOPIAN THE LARGEST AIRLINE IN AFRICA FACT SHEET 2 Ethiopian Background Information Founded E December 21, 1945 Starting date of operation E April 08, 1946 Ownership E Government of Ethiopia (100%) Head Office E Bole International Airport, P .O. Box 1755 Addis Ababa, Ethiopia Fax: (+ 251)11661 1474 Reservations E Tel: (+251) 11 665 6666 Website E http://www.ethiopianairlines.com Chief Executive Officer E Mr. Tewolde GebreMariam Overview Ethiopian Airlines (Ethiopian) is the leading and most profitable airline in Africa. Tel: (+251-11) 517 8407/8656/8907/8165/8529/8913 Fax: (+251-11) 661 1474 E-mail:CorporateCommunication@ethiopianairlines.com www.twitter.com/flyethiopian www.youtube.com/flyethiopian www.facebook.com/ethiopianairlines.com http://plus.google.com/+ethiopianairlinescom http://instagram.com/fly.ethiopian www.ethiopianairlines.com Contact us on: Best Cargo Airline of the Year from Africa at

In [21]:
print(output["draft_speech"])

Ladies and Gentlemen,

I stand before you today to speak about a company that has not only made its mark in Africa but has also made a significant impact on the global aviation industry. This company is none other than Ethiopian Airlines, the leading and most profitable airline in Africa. 

Founded on December 21, 1945, and commencing operations on April 8, 1946, Ethiopian Airlines has been a beacon of resilience and progress. With its head office at Bole International Airport in Addis Ababa, Ethiopia, the airline is wholly owned by the Government of Ethiopia. Under the astute leadership of our Chief Executive Officer, Mr. Tewolde GebreMariam, Ethiopian Airlines has soared to great heights.

Ethiopian Airlines is not just an airline; it is "Africa's Link to the World." Each year, more than 1.5 million people choose to fly with us to 22 domestic and 44 international destinations across four continents. Our commitment to excellence has been recognized globally, with Ethiopian Airlines be

In [ ]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory) 

if __name__ == "__main__":
    initial_state = {
        "company_name": "sintec ethiopia plc",
        "country_name": "Ethiopia",
        "revision_number": 1,
        "max_revisions": 2,
        "thread_id": "session_001",  
        "company_info": "",
        "country_info": "",
        "draft_speech": "",
        "retrieved_info": []
    }

    output = graph.invoke(initial_state, config={"configurable": {"thread_id": "session_001"}})
    print(output)


{'company_name': 'sintec ethiopia plc', 'country_name': 'Ethiopia', 'company_info': 'SINTEC ETHIOPIA (plc) is one of the first Companies in Ethiopia established in 1989 among a consortium of seven sister companies. These group of companies include SINTEC ETHIOPIA (plc) Metaferia Consulting Engineers (MCE), African Business & Development Consultants (ABD - Consult), ESSET P.L.C. and SINTEC ENERGY, SINTEC MOTORS and ESSET Constructions.\nWelcome To Sintec Ethiopia! SINTEC ETHIOPIA (plc) is one of the first Companies in Ethiopia established in 1989 among a consortium of seven sister companies. These group of companies include SINTEC ETHIOPIA (plc) Metaferia Consulting Engineers (MCE), African Business & Development Consultants (ABD - Consult), ESSET P.L.C. and SINTEC ENERGY, SINTEC MOTORS and ESSET Constructions.\nSINTEC ETHIOPIA PLC is a private Electra-mechanical engineering company established in 1989. Our vision is to enable technology transfer to Ethiopia and the African continent. .

In [19]:
print(output["draft_speech"])

Ladies and Gentlemen,

I stand before you today, not just as a representative of SINTEC ETHIOPIA PLC, but as a proud advocate for the immense potential that lies within the heart of Africa, in the vibrant nation of Ethiopia. SINTEC ETHIOPIA, established in 1989, is one of the pioneering companies in Ethiopia, part of a consortium of seven sister companies, all dedicated to the growth and development of our beloved nation.

Our vision at SINTEC ETHIOPIA is to enable technology transfer to Ethiopia and the African continent. We are a private Electra-mechanical engineering company, partnering with global industry leaders to import and supply industrial goods to our clientele. Our annual revenue is a testament to our commitment and the trust our clients place in us.

But why Ethiopia, you may ask? Ethiopia, located in the Horn of Africa, has emerged as a promising destination for international business. With a rapidly growing economy, strategic geographical location, and a population of ov